# Aula 2: Experimentação e MVP de Modelos

## Objetivos de Aprendizagem
- Configurar e usar MLFlow para rastreamento de experimentos
- Experimentar com diferentes algoritmos e hiperparâmetros
- Comparar resultados de múltiplos experimentos
- Criar um MVP (Minimum Viable Product) de modelo

## Exercício Prático
Você trabalhará com um dataset de classificação e realizará experimentos com diferentes modelos.

## 1. Configuração do Ambiente

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import mlflow
import mlflow.sklearn
import warnings
warnings.filterwarnings('ignore')

## 2. Carregamento e Preparação dos Dados

In [ ]:
# Carregar dataset
wine = load_wine()
X = pd.DataFrame(wine.data, columns=wine.feature_names)
y = wine.target

print(f"Shape dos dados: {X.shape}")
print(f"Classes: {np.unique(y)}")
X.head()

In [ ]:
# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Normalizar dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Treino: {X_train.shape}, Teste: {X_test.shape}")

## 3. Configuração do MLFlow

### Tarefa 1: Configure o MLFlow para rastrear seus experimentos

In [ ]:
# Configurar experimento do MLFlow
mlflow.set_experiment("wine_classification_experiments")

# Iniciar UI do MLFlow (opcional - executar em terminal separado)
# mlflow ui

## 4. Experimento 1: Regressão Logística

### Tarefa 2: Treine um modelo de Regressão Logística e registre no MLFlow

In [ ]:
with mlflow.start_run(run_name="logistic_regression_baseline"):
    # Parâmetros
    max_iter = 1000
    C = 1.0
    
    # Treinar modelo
    model = LogisticRegression(max_iter=max_iter, C=C, random_state=42)
    model.fit(X_train_scaled, y_train)
    
    # Fazer previsões
    y_pred = model.predict(X_test_scaled)
    
    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    
    # Registrar parâmetros
    mlflow.log_param("model_type", "LogisticRegression")
    mlflow.log_param("max_iter", max_iter)
    mlflow.log_param("C", C)
    
    # Registrar métricas
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    
    # Registrar modelo
    mlflow.sklearn.log_model(model, "model")
    
    print(f"Logistic Regression - Accuracy: {accuracy:.4f}, F1: {f1:.4f}")

## 5. Experimento 2: Árvore de Decisão

### Tarefa 3: Experimente com diferentes profundidades de árvore

In [ ]:
# Experimentar com diferentes profundidades
max_depths = [3, 5, 10, None]

for max_depth in max_depths:
    with mlflow.start_run(run_name=f"decision_tree_depth_{max_depth}"):
        # Treinar modelo
        model = DecisionTreeClassifier(max_depth=max_depth, random_state=42)
        model.fit(X_train_scaled, y_train)
        
        # Fazer previsões
        y_pred = model.predict(X_test_scaled)
        
        # Calcular métricas
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        
        # Registrar parâmetros e métricas
        mlflow.log_param("model_type", "DecisionTree")
        mlflow.log_param("max_depth", max_depth if max_depth else "None")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("f1_score", f1)
        
        # Registrar modelo
        mlflow.sklearn.log_model(model, "model")
        
        print(f"Decision Tree (depth={max_depth}) - Accuracy: {accuracy:.4f}, F1: {f1:.4f}")

## 6. Experimento 3: Random Forest

### Tarefa 4: Otimize hiperparâmetros do Random Forest

In [ ]:
# Experimentar com diferentes configurações
configs = [
    {"n_estimators": 50, "max_depth": 5},
    {"n_estimators": 100, "max_depth": 10},
    {"n_estimators": 200, "max_depth": None},
]

for config in configs:
    with mlflow.start_run(run_name=f"random_forest_{config['n_estimators']}_trees"):
        # Treinar modelo
        model = RandomForestClassifier(
            n_estimators=config["n_estimators"],
            max_depth=config["max_depth"],
            random_state=42
        )
        model.fit(X_train_scaled, y_train)
        
        # Fazer previsões
        y_pred = model.predict(X_test_scaled)
        
        # Calcular métricas
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        
        # Registrar parâmetros e métricas
        mlflow.log_param("model_type", "RandomForest")
        mlflow.log_param("n_estimators", config["n_estimators"])
        mlflow.log_param("max_depth", config["max_depth"] if config["max_depth"] else "None")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("f1_score", f1)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        
        # Registrar modelo
        mlflow.sklearn.log_model(model, "model")
        
        print(f"Random Forest ({config['n_estimators']} trees) - Accuracy: {accuracy:.4f}, F1: {f1:.4f}")

## 7. Comparação de Experimentos

### Tarefa 5: Compare os resultados de todos os experimentos

In [ ]:
# Buscar todos os runs do experimento
experiment = mlflow.get_experiment_by_name("wine_classification_experiments")
runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id])

# Visualizar resultados
print("\n=== COMPARAÇÃO DE EXPERIMENTOS ===")
print(runs[['tags.mlflow.runName', 'metrics.accuracy', 'metrics.f1_score']].sort_values(
    by='metrics.accuracy', ascending=False
))

## 8. Seleção do Melhor Modelo (MVP)

### Tarefa 6: Selecione e registre o melhor modelo

In [ ]:
# Encontrar o melhor run
best_run = runs.loc[runs['metrics.accuracy'].idxmax()]
best_run_id = best_run['run_id']

print(f"\nMelhor Modelo:")
print(f"Run Name: {best_run['tags.mlflow.runName']}")
print(f"Accuracy: {best_run['metrics.accuracy']:.4f}")
print(f"F1 Score: {best_run['metrics.f1_score']:.4f}")
print(f"Run ID: {best_run_id}")

## 9. Exercícios Adicionais

### Desafios para Praticar:

1. **Adicione mais modelos**: Experimente com SVM, KNN ou Gradient Boosting
2. **Validação Cruzada**: Implemente cross-validation e registre as métricas médias
3. **Feature Importance**: Para modelos baseados em árvores, registre a importância das features
4. **Gráficos**: Crie gráficos de comparação e salve como artifacts no MLFlow
5. **Tags**: Use tags do MLFlow para organizar seus experimentos (ex: "production_candidate")

### Questões para Reflexão:

1. Qual modelo teve o melhor desempenho? Por quê?
2. Existe evidência de overfitting em algum modelo?
3. Como você escolheria entre um modelo com maior accuracy e um com melhor F1 score?
4. Quais outros experimentos você faria antes de colocar o modelo em produção?

## 10. Acessando o MLFlow UI

Para visualizar seus experimentos graficamente:

```bash
mlflow ui
```

Depois acesse http://localhost:5000 no seu navegador.